## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# config.py file is located in the parent directory of the current folder.
# Add the parent directory to sys.path (sys.path contains PYTHONPATH) 
import sys
sys.path.append('../')

# Import API key 
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

# Load the csv file from parent directory's Weather_Database folder 
weather_file = os.path.join("..","Weather_Database","WeatherPy_Database.csv")
city_data_df = pd.read_csv(weather_file)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Grindavik,IS,63.8424,-22.4338,49.95,81,75,5.75,broken clouds
1,1,Tonantins,BR,-2.8731,-67.8022,92.03,44,89,3.31,overcast clouds
2,2,Nauchnyy Gorodok,RU,53.4203,83.5210,43.66,52,0,8.95,clear sky
3,3,East London,ZA,-33.0153,27.9116,63.54,94,40,9.22,scattered clouds
4,4,Kumylzhenskaya,RU,49.8809,42.5898,66.97,97,100,6.40,moderate rain


In [3]:
# Check for data types of columns to be mapped: they should be float or int
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# Check for null values in dataframe
city_data_df.isnull().count()

City_ID                690
City                   690
Country                690
Lat                    690
Lng                    690
Max Temp               690
Humidity               690
Cloudiness             690
Wind Speed             690
Current Description    690
dtype: int64

In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 86


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Ribeira Grande,PT,38.5167,-28.7000,77.40,78,20,13.80,few clouds
11,11,Bonthe,SL,7.5264,-12.5050,78.08,85,100,9.15,overcast clouds
15,15,General Pico,AR,-35.6566,-63.7568,79.48,19,100,24.90,overcast clouds
20,20,Caravelas,BR,-17.7125,-39.2481,79.38,61,28,8.88,scattered clouds
21,21,Jamestown,US,42.0970,-79.2353,80.02,46,81,4.00,broken clouds
22,22,Mitu,CO,1.1983,-70.1733,79.68,86,99,3.13,overcast clouds
23,23,Georgetown,MY,5.4112,100.3354,78.89,92,20,2.30,few clouds
26,26,Kalmunai,LK,7.4167,81.8167,79.66,78,12,6.55,few clouds
27,27,Guanare,VE,9.0418,-69.7421,83.23,78,100,2.66,overcast clouds
29,29,Ponta Do Sol,PT,32.6667,-17.1000,78.71,93,0,5.44,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                206
City                   206
Country                205
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                205
City                   205
Country                205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Ribeira Grande,PT,77.40,few clouds,38.5167,-28.7000,
11,Bonthe,SL,78.08,overcast clouds,7.5264,-12.5050,
15,General Pico,AR,79.48,overcast clouds,-35.6566,-63.7568,
20,Caravelas,BR,79.38,scattered clouds,-17.7125,-39.2481,
21,Jamestown,US,80.02,broken clouds,42.0970,-79.2353,
22,Mitu,CO,79.68,overcast clouds,1.1983,-70.1733,
23,Georgetown,MY,78.89,few clouds,5.4112,100.3354,
26,Kalmunai,LK,79.66,few clouds,7.4167,81.8167,
27,Guanare,VE,83.23,overcast clouds,9.0418,-69.7421,
29,Ponta Do Sol,PT,78.71,clear sky,32.6667,-17.1000,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.isnull().count()

City                   205
Country                205
Max Temp               205
Current Description    205
Lat                    205
Lng                    205
Hotel Name             205
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name']!="")]
clean_hotel_df.count()

City                   193
Country                193
Max Temp               193
Current Description    193
Lat                    193
Lng                    193
Hotel Name             193
dtype: int64

In [13]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Ribeira Grande,PT,77.40,few clouds,38.5167,-28.7000,Quinta da Meia Eira
11,Bonthe,SL,78.08,overcast clouds,7.5264,-12.5050,Bonthe Holiday Village
15,General Pico,AR,79.48,overcast clouds,-35.6566,-63.7568,Euskadi Hotel
20,Caravelas,BR,79.38,scattered clouds,-17.7125,-39.2481,Pousada dos Navegantes
21,Jamestown,US,80.02,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
    <dt>Hotel Name:</dt><dd>{Hotel Name}</dd>
    <dt>City:</dt><dd>{City}</dd>
    <dt>Country:</dt><dd>{Country}</dd>    
    <dt>Current Weather:</dt><dd>{Current Description}</dd>
    <dt>Max Temp:</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 

# Get the fig object
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Set the marker layer
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Add layer to fig
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))